# Assignment 1 - solution

In [8]:
# Some packages you might need in the following

from nose.tools import assert_equal, assert_raises  # Some functions to produce some of the checks before

import numpy as np
import matplotlib.pylab as plt
from numpy.random import default_rng
from scipy.optimize import minimize 

?minimize


## Question 1. [40 points]

An investor has utility function $$u(x) = \begin{cases} \frac{|x|^{p}}{x} & \text{ if } x\neq 0;\\ 0 & \text{ if } x=0\end{cases},$$ for $p>1$.

- 1.1. Implement this utility function [10 points]

In [9]:
def u(x, p): 
    """
    Compute the utility function $u$ defined above on a real 'x' and a given fixed value 'p'>1
    """
    ### BEGIN SOLUTION
    # Put correct code here.  This code is removed for the student version, but is used
    # to confirm that your tests are valid.
    
    try: 
        x= float(x)
    except:
        raise TypeError('The variable x is not a real')
    
    
    try: 
        p=float(p)
    except:
        raise TypeError('The variable p is not a real')
    
    if p<=1:
        raise ValueError('The variable p must be larger than one')
    
    if x==0:
        return 0
    return abs(x)**p/x
    
    ### END SOLUTION

- 1.2. Define now a function 'u_vec' that receives a numpy array or a list and a real p larger than one and returns an array with the utility function above applied entry-wise [10 points].

In [10]:
def u_vec(xvec, p):  
    """
    Receives a numpy array or a list 'xvec' and a real 'p'>1 and returns an array with the utility function above applied entry-wise
    """
    ### BEGIN SOLUTION
    # Put correct code here.  This code is removed for the student version, but is used
    # to confirm that your tests are valid.
    
    try:
        xvec=np.asarray(xvec)
    except:
        raise TypeError('xvec is not an object that can be treated like an array')
    
    return np.array([u(x,p) for x in xvec])
    ### END SOLUTION

- 1.3 Create a function that calculates the coefficient of absolute risk aversion of this function for a given real 'x' [10 points]


In [11]:
def ara(x,p):
    """
    Compute the coefficient of absolute risk aversion 
    """
    ### BEGIN SOLUTION
    # Put correct code here.  This code is removed for the student version, but is used
    # to confirm that your tests are valid.
    
    try: 
        x= float(x)
    except:
        raise TypeError('The variable x is not a real')
    
    
    try: 
        p=float(p)
    except:
        raise TypeError('The variable p is not a real')
    
    if p<=1:
        raise ValueError('The variable p must be larger than one')
    
    
    if x==0 and (p!= 2):
        # The utility fucntion is not twice differenciable at zero if the coefficient is not 2.
        raise ValueError('The coefficient of risk aversion is not defined for x=0 and p=',p)
    if p==2:
        return 0
    return (2-p)/x
    ### END SOLUTION

- 1.4. Comment on the risk behaviour of an investor with this function [10 points]


Depends on the coefficient $p$: If $p=2$, then the utility function is just linear. If $p>2$, then the function is risk-averse for positive wealth and risk-seeking for negative wealth. If $1<p<2$ then is the opposite.

## Question 2. [20 points]

Consider a market model in one-period with three assets: a risk free asset with unit return, and two risky assets with log normal returns

$R^1_1\sim \exp(N_1), \qquad R^2_1\sim \exp(N_2)$
where $N_1\sim \mathcal N(m_1,\sigma_1^2)$, $N_2\sim \mathcal N(m_2,\sigma_2^2) $ and the Gaussian random variables are correlated $$\mathrm{cor}(N_1,N_2)=\rho.$$ 



- 2.1. Explain if this market is complete [10 points]


The market is incomplete. Note that in one period the set of all attainable claims is only the linear combination of the payoffs of the three assets. But this does not cover integrable wealth profiles (like $(R_1^1)^2$ or $\max\{R^1_1,R^2_1\}$ )

- 2.2. Create a function that returns a sample of a given size of (${R}^1_1$, ${R}_1^2$). [10 points]


In [12]:
def make_sample(n, m1,m2, sigma1,sigma2,rho): 
    """
    Create a sample of (${R}^1_1$, ${R}_1^2$) of size 'n', where the other parameters are defined as given. Returns a tuple (R1_sample,R2_sample)
    """
    ### BEGIN SOLUTION
    # Put correct code here.  This code is removed for the student version, but is used
    # to confirm that your tests are valid.
    
    gen = default_rng()

    z1 = gen.standard_normal(n)
    z2 = gen.standard_normal(n)
    
    
    R1_sample= np.exp(sigma1 * z1+ m1 )
    R2_sample= np.exp( sigma2*(rho*z1 + ((1-rho**2)**0.5)*z2)+m2)        
    
    return R1_sample,R2_sample
    
    ### END SOLUTION
    
    
    

## Question 3 [40 points]

- 3.1. Let $\pi$ be a strategy (in terms of percentages) in a market with three assets (one risk-free and two risky as in Question 2). Let $\hat \pi$ be the components of the strategy on the risky assets. Define a function that returns the expected utility for an investor with a utility function as in Question 1 and initial wealth $w_0$, after applying the strategy represented by $\hat \pi$ (recall that the investment on the risk-less asset can be deduced from the budget constraint). The function should also receive a sample of $\hat R_1$. [20 points]

In [13]:
def u_exp(pi_hat, w0,R_hat,p):  
    """
    Returns the expected utility of Q1 (with parameter 'p') of the strategy characterised by 'pi_hat', from a sample 'R_hat'
    """
    ### BEGIN SOLUTION
    
    W = w0*((1-pi_hat.sum()) + np.asarray(R_hat).T@pi_hat)
    return u_vec(W,p).mean()
    
    
    ### END SOLUTION

- 3.2. Write a function that approximates the solution of the optimal investment problem in the market defined in Question 2, for an investor as described in 3.1..  The function should return a couple, the strategy (as a vector of weights for the two risky assets) and the value of the utility for the optimal portfolio. If no solution is found, 'None' should be returned. [20 points]

In [14]:
def optimal_investment(w0,m1,m2,sigma1,sigma2,rho,p):  # modify function name and parameters
    """
    Return pi_hat and expected utility of optimal strategy for investor with utility function with param 'p', and market with params. 'm1,m2,sigma1,sigma2,rho'
    """
    ### BEGIN SOLUTION
    
    
    n = 640000
    R_hat = np.array(make_sample(n,m1,m2,sigma1,sigma2,rho))
    pi_hat = np.zeros(2)
    
    sol = minimize( lambda x: -u_exp(x,w0,R_hat,p), pi_hat )
    
    
    if sol['success']:
        return  sol['x'], -1*sol['fun']
    return None
    
    
    ### END SOLUTION

In [15]:

assert_equal(len(optimal_investment(100,0.01,0.01,0.1,0.2,0,1.5)[0]),2)
aux_f = lambda x: np.append(x[0],x[1])
np.testing.assert_allclose(aux_f(optimal_investment(100,0.01,0.02,0.16,0.16,-0.5,1.5)), np.array([2.75, 3.25,10.5 ]), rtol=5e-1, atol=2e-1)

